In [4]:
# Step 0) Handle API Key's Securely
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Step 1) Use Cohere's Embed Model to create text embeddings

# Setup Cohere
import cohere
co = cohere.Client(os.environ.get('COHERE_API_KEY'))

to_embed = [
    "I've heard USC is called the University of Spoiled Children. Do you find this to be true?",
    "Not at all! During my time at USC, I have always felt surrounded by an extremely diverse student community. Students come from all types of backgrounds and this really adds to a cohesive and enriching student dynamic. That being said, I think that regardless of a USC student's socio-economic background, everyone at USC works hard and comes together to explore academic and extracurricular interests."
]

In [6]:
# Embed Sentences
# def get_embedding(sent: str) -> :

sentence_embeddings = co.embed(
    texts=to_embed,
    model="embed-english-v3.0",
    input_type="search_document",
    truncate='END',
    embedding_types=['float']
).embeddings.float

In [7]:
# Import Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

/Users/kyleng/B_Organized/dev/nlp/rag/RAG-Ambassador-Training/.venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
index_name = "cohere-test-embeddings"
existing_indexes = [
    index["name"] for index in pc.list_indexes()
]
index_exists = index_name in existing_indexes

In [9]:
print (index_name in pc.list_indexes())

print(pc.list_indexes())

False
{'indexes': [{'dimension': 8,
              'host': 'quickstart-first-1efl3et.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'euclidean',
              'name': 'quickstart-first',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [10]:
# Create an Index
if not index_exists:
    pc.create_index(
        name=index_name,
        dimension=8,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )